In [66]:
import os
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image, ImageDraw, ImageFilter

Renaming

In [67]:
# dataset_img = sorted(os.listdir('./NTNU_Herbarium_Segmentation_V2-5/train/images'))
# dataset_mask = sorted(os.listdir('./NTNU_Herbarium_Segmentation_V2-5/train/labels'))

# for idx, (img, mask) in enumerate(zip(dataset_img, dataset_mask)):
#     new_name = img.split('.')[0].replace('_jpg', '')
#     os.rename(f'./NTNU_Herbarium_Segmentation_V2-5/train/images/{img}', f'./NTNU_Herbarium_Segmentation_V2-5/train/images/{new_name}.png')
#     os.rename(f'./NTNU_Herbarium_Segmentation_V2-5/train/labels/{mask}', f'./NTNU_Herbarium_Segmentation_V2-5/train/labels/{new_name}.png')

Only selecting a few (manually chosen)

In [68]:
# selected = ['1701313016_a', '1701329739', '1701297370', '1701252546_a', '1701329537', '1701328720_a', '1701562666', '1701306813', '1701289461', '1697947089', '1701221440', '1701221792', '1701248131']

# os.makedirs('./to_augment/images', exist_ok=True)
# os.makedirs('./to_augment/labels', exist_ok=True)

# for img in selected:
#     os.rename(f'./NTNU_Herbarium_Segmentation_V2-5/train/images/{img}.png', f'./to_augment/images/{img}.png')
#     os.rename(f'./NTNU_Herbarium_Segmentation_V2-5/train/labels/{img}.png', f'./to_augment/labels/{img}.txt')

Cropping out the masks and saving them in binary form

In [162]:
def get_mask_points(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()
    lines = [line.strip().split(' ')[1:] for line in lines] # remove the first element which is the class label
    points = [[[float(line[i]), float(line[i+1])] for i in range(0, len(line), 2)] for line in lines] # convert to list of list of points where x and y are combined
    return points

def find_dominant_color(image: Image, k: int = 5) -> tuple:
    
    # Convert image to numpy array
    img_array = np.array(image)
    
    # Reshape it to a list of RGB values
    img_vector = img_array.reshape((-1, 3))
    
    # Run k-means on the pixel colors
    kmeans = KMeans(n_clusters=k, random_state=0).fit(img_vector)
    
    # Get the dominant color
    dominant_color = kmeans.cluster_centers_[np.argmax(np.bincount(kmeans.labels_))]
    
    # Create a mask for pixels within a certain distance from the dominant color
    distances = np.sqrt(np.sum((img_vector - dominant_color) ** 2, axis=1))
    mask = distances < np.std(distances)
    
    # Turn the dominant color range to white
    img_vector[mask] = [255, 255, 255]
    result_img_array = img_vector.reshape(img_array.shape)

    return dominant_color, result_img_array

def crop_out_image(image_path, points):

    image = Image.open(image_path) # open the image
    _, white_bg = find_dominant_color(image) # find the dominant color
    white_bg = Image.fromarray(white_bg) # convert to image
    image = white_bg.convert('RGBA') # convert to RGBA

    for idx, point in enumerate(points):

        mask = Image.new('L', image.size, 0) # create a mask
        draw = ImageDraw.Draw(mask)

        unnormalized_points = [(int(x * image.width), int(y * image.height)) for [x, y] in point] # un-normalize the points

        draw.polygon(unnormalized_points, fill=255)

        transparent_bg = Image.new('RGBA', image.size, (0, 0, 0, 0)) # create a white background

        result = Image.composite(image, transparent_bg, mask) # crop out the image

        # Turn all the white pixels to transparent
        data = np.array(result)
        white = np.all(data[:, :, :3] == 255, axis=-1)
        data[white, -1] = 0
        result = Image.fromarray(data)

        name = os.path.basename(image_path).split('.')[0]
        result.save(f'{name}_object_{idx}.png', 'PNG')

def remove_objects_from_image(image_path, points):
    image = Image.open(image_path) # open the image
    _, result_img_array = find_dominant_color(image) # find the dominant color
    image = Image.fromarray(result_img_array) # convert the numpy array to image

    for point in points:

        unnormalized_points = [(int(x * image.width), int(y * image.height)) for [x, y] in point] # un-normalize the points

        ImageDraw.Draw(image).polygon(unnormalized_points, fill='white') # fill the polygon with white color

    name = os.path.basename(image_path).split('.')[0]
    image.save(f'{name}_background.png', 'PNG')

def rotate_point(cx, cy, angle, px, py):
    radians = np.radians(angle)
    cos_angle = np.cos(radians)
    sin_angle = np.sin(radians)
    px -= cx
    py -= cy
    xnew = px * cos_angle - py * sin_angle
    ynew = px * sin_angle + py * cos_angle
    px = xnew + cx
    py = ynew + cy
    return px, py

def update_points(points, perc_offset_x, perc_offset_y, rotation_angle, image_width, image_height):
    # Calculate pixel offsets from percentages
    offset_x = perc_offset_x * image_width / 100
    offset_y = perc_offset_y * image_height / 100

    # Image center as rotation center
    rotation_center = (image_width / 2, image_height / 2)

    # Rotate and then translate points
    updated_points = []
    for x, y in points:
        new_x, new_y = rotate_point(rotation_center[0], rotation_center[1], rotation_angle, x, y)
        new_x += offset_x
        new_y += offset_y
        updated_points.append((new_x, new_y))
    return updated_points

In [163]:
image_name = '1701313016_a'
original_points = get_mask_points(f'./to_augment/labels/{image_name}.txt')

original_image = Image.open(f'./to_augment/images/{image_name}.png')
image_width, image_height = original_image.size

remove_objects_from_image(f'./to_augment/images/{image_name}.png', original_points)
crop_out_image('./to_augment/images/1701313016_a.png', original_points)

len(original_points)

5

Manually update the settings dictionary with the changes you made in the photo-editing software

In [164]:
settings = {
    '0': {
        'x_perc_offset': -3.13,
        'y_perc_offset': 41.47,
        'rotation_angle': 0
    },
    '1': {
        'x_perc_offset': 3.75,
        'y_perc_offset': 39.38,
        'rotation_angle': 4.88
    },
    '2': {
        'x_perc_offset': -0.23,
        'y_perc_offset': -34.45,
        'rotation_angle': 0
    },
    '3': {
        'x_perc_offset': 0.68,
        'y_perc_offset': -41.39,
        'rotation_angle': 2.43
    },
}

In [167]:
os.makedirs('./augmented/images', exist_ok=True)
os.makedirs('./augmented/labels', exist_ok=True)

for idx, point in enumerate(original_points):
    if idx >= 4:
        break
    unnormalized_points = [(int(x * image_width), int(y * image_height)) for [x, y] in point]
    
    x_perc_offset = settings[str(idx)]['x_perc_offset']
    y_perc_offset = settings[str(idx)]['y_perc_offset']
    rotation_angle = settings[str(idx)]['rotation_angle']

    new_points = update_points(unnormalized_points, x_perc_offset, y_perc_offset, rotation_angle, image_width, image_height)

    with open(f'./augmented/labels/{image_name}_augmented_0.txt', 'a') as f:
        f.write('0 ')
        for x, y in new_points:
            f.write(f'{x/image_width} {y/image_height}')
            f.write(' ')
        f.write('\n')